In [1]:
import numpy as np
import pandas as pd

In [2]:
alldata = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick202004final.xlsx")
train1data = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick202004train1final.xlsx")
train2data = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick202004train2final.xlsx")
train3data = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick202004train3final.xlsx")

In [3]:
data1 = alldata.sort_values("LocationID")
data1 = data1.reset_index(drop=True)

In [4]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(data1[['pop_density','commercial_density','road_density','X','Y']])
data1[['pop_density','commercial_density','road_density','X','Y']] = scaler.transform(data1[['pop_density','commercial_density','road_density','X','Y']])


In [5]:
X = np.array(data1[["X"]])
Y = np.array(data1[["Y"]])
coords = np.hstack([X,Y])

t = np.array(data1[["time"]])

y = np.array(data1[["Freq"]])

pop = np.array(data1[["pop_density"]])
com = np.array(data1[["commercial_density"]])
roa = np.array(data1[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.model import GTWR

results1 = GTWR(coords, t, y, x,80.0 ,0.01 ,kernel='bisquare').fit()

results1.R2

0.7723778108615096

In [6]:
coords = results1.coords
coords = pd.DataFrame(coords)

time = results1.t
time = pd.DataFrame(time)

betas = results1.betas
betas = pd.DataFrame(betas)

tvalues = results1.tvalues
tvalues = pd.DataFrame(tvalues)

final_result = pd.concat([time,betas,tvalues,coords], axis = 1)
final_result.columns=['time','beta0','POP_beta','COM_beta','ROA_beta','t_beta0','t_POP_beta','t_COM_beta','t_ROA_beta','X','Y']

time_data = final_result.sort_values("time")

data2 = pd.merge(data1, time_data, left_on=['time','X','Y'], right_on=['time','X','Y'] , how='left')
data2 = data2.sort_values("time")
data2 = data2.reset_index(drop=True)
data2.drop(['Freq','pop_density','commercial_density','road_density'],axis=1,inplace = True)

data3 = pd.DataFrame()
for j in range(len(data2)):
    if abs(data2.iloc[j,8]) > 0 and abs(data2.iloc[j,9]) > 0 and abs(data2.iloc[j,10]) > 0 and abs(data2.iloc[j,11]) > 0 :
        a = data2.iloc[j,:]
        data3 = pd.concat([data3, a], axis=1, ignore_index = True)

data3 = data3.T
beta = data3.groupby('time').mean()
beta.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick202004beta.csv',index=True,sep=',')

data = data3.round(6)

beta1 = data.groupby('LocationID')['POP_beta'].mean()
beta1.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick202004beta1.csv',index=True,header = 0,sep=',')

beta2 = data.groupby('LocationID')['COM_beta'].mean()
beta2.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick202004beta2.csv',index=True,header = 0,sep=',')

beta3 = data.groupby('LocationID')['ROA_beta'].mean()
beta3.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick202004beta3.csv',index=True,header = 0,sep=',')

In [7]:
betas = results1.betas
coefficients = pd.DataFrame(betas)
coefficients.describe()

,0,1,2,3
count,25032.000000,25032.000000,25032.000000,25032.000000
mean,10.350621,1.203110,1.442195,0.408526
std,12.724737,5.331296,4.576755,4.577861
min,-20.322564,-21.571909,-18.660685,-28.492131
25%,2.017783,-0.780799,-0.442191,-1.204774
50%,3.347958,0.125220,0.282938,-0.048407
75%,17.051475,1.787851,2.421814,1.445771
max,69.943104,43.633399,31.751292,19.909543


In [16]:
tvalues = results1.tvalues
t = pd.DataFrame(tvalues)

In [24]:
data = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick202004final.xlsx")
data1 = data.iloc[:,0:2]
data2 = pd.concat([data1, coefficients,t], axis = 1)
data2.columns=['time','LocationID','beta0','POP_beta','COM_beta','ROA_beta','t_beta0','t_POP_beta','t_COM_beta','t_ROA_beta']
data3 = pd.DataFrame()
for j in range(len(data2)):
    if abs(data2.iloc[j,6]) > 2 and abs(data2.iloc[j,7]) > 2 and abs(data2.iloc[j,8]) > 2 and abs(data2.iloc[j,9]) > 2 :
        a = data2.iloc[j,:]
        data3 = pd.concat([data3, a], axis=1, ignore_index = True)

data3 = data3.T
beta = data3.groupby('time').mean()
beta.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick202004beta.csv',index=True,sep=',')

In [25]:
data = data3.round(6)

In [26]:
beta1 = data.groupby('LocationID')['POP_beta'].mean()
beta1.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick202004beta1.csv',index=True,header = 0,sep=',')

In [28]:
beta2 = data.groupby('LocationID')['COM_beta'].mean()
beta2.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick202004beta2.csv',index=True,header = 0,sep=',')

In [29]:
beta3 = data.groupby('LocationID')['ROA_beta'].mean()
beta3.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick202004beta3.csv',index=True,header = 0,sep=',')

In [11]:
final = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick202004final.xlsx")
hour = final.iloc[:,0:2]
data = pd.concat([hour, coefficients], axis = 1)
beta = data.groupby('time').mean()
beta.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick202004beta.csv',index=True,sep=',')

In [12]:
final = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick202004final.xlsx")
hour = final.iloc[:,0:2]
data = pd.concat([hour, coefficients], axis = 1)
data.columns=['time','LocationID','beta0','POP_beta','COM_beta','ROA_beta']
data = data.round(6)

In [13]:
beta1 = data.groupby('LocationID')['POP_beta'].mean()
beta1.to_csv(r'E:\\Capstone Project\\Data\\paper\\pick202004beta1.csv',index=True,header = 0,sep=',')

In [33]:
X = np.array(data1[["X"]])
Y = np.array(data1[["Y"]])
coords = np.hstack([X,Y])

t = np.array(data1[["time"]])

y = np.array(data1[["Freq"]])

pop = np.array(data1[["pop_density"]])
com = np.array(data1[["commercial_density"]])
roa = np.array(data1[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.sel import search_GTWRparameter

sel = search_GTWRparameter(coords, t, y, x, kernel = 'bisquare')

bw, tau = sel.search(verbose = True)

bw:  9591.0 , tau:  0.2 , score:  212473.33169064467
bw:  5946.0 , tau:  0.2 , score:  212322.75577537267
bw:  3693.0 , tau:  0.2 , score:  211953.8693995708
bw:  2301.0 , tau:  0.2 , score:  211053.75727800356
bw:  1440.0 , tau:  0.2 , score:  209329.2678645656
bw:  909.0 , tau:  0.2 , score:  206787.91679011605
bw:  580.0 , tau:  0.2 , score:  203512.59761928406
bw:  377.0 , tau:  0.2 , score:  200415.47006140952
bw:  251.0 , tau:  0.2 , score:  197800.0719899826
bw:  174.0 , tau:  0.2 , score:  196854.38320744372
bw:  126.0 , tau:  0.2 , score:  196540.44778541557
bw:  96.0 , tau:  0.2 , score:  195660.91522446088
bw:  78.0 , tau:  0.2 , score:  195587.15567063956
bw:  78.0 , tau:  0.2 , score:  195587.15567063956
bw:  78.0 , tau:  3.8 , score:  195587.15567063956
bw:  78.0 , tau:  0.2 , score:  195587.15567063956
bw:  80.0 , tau:  0.2 , score:  195548.82075275638
bw:  80.0 , tau:  0.2 , score:  195548.82075275638
bw:  80.0 , tau:  0.2 , score:  195548.82075275638


In [34]:
data2 = train1data.sort_values("LocationID")
data2 = data2.reset_index(drop=True)


from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(data2[['pop_density','commercial_density','road_density','X','Y']])
data2[['pop_density','commercial_density','road_density','X','Y']] = scaler.transform(data2[['pop_density','commercial_density','road_density','X','Y']])

In [35]:
X = np.array(data2[["X"]])
Y = np.array(data2[["Y"]])
coords = np.hstack([X,Y])

t = np.array(data2[["time"]])

y = np.array(data2[["Freq"]])

pop = np.array(data2[["pop_density"]])
com = np.array(data2[["commercial_density"]])
roa = np.array(data2[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.sel import search_GTWRparameter

sel = search_GTWRparameter(coords, t, y, x, kernel = 'bisquare')

bw, tau = sel.search(verbose = True)

bw:  5728.0 , tau:  0.2 , score:  125537.39787504295
bw:  3559.0 , tau:  0.2 , score:  125302.39443255882
bw:  2218.0 , tau:  0.2 , score:  124714.11404995823
bw:  1389.0 , tau:  0.2 , score:  123616.14087210444
bw:  877.0 , tau:  0.2 , score:  121987.45648196587
bw:  560.0 , tau:  0.2 , score:  119915.14990219801
bw:  365.0 , tau:  0.2 , score:  118039.2131896007
bw:  244.0 , tau:  0.2 , score:  116426.56032078073
bw:  169.0 , tau:  0.2 , score:  115857.21064593595
bw:  123.0 , tau:  0.2 , score:  115638.0159883876
bw:  94.0 , tau:  0.2 , score:  115073.04364268018
bw:  77.0 , tau:  0.2 , score:  115006.13055760202
bw:  77.0 , tau:  0.2 , score:  115006.13055760202
bw:  77.0 , tau:  3.8 , score:  115006.13055760202
bw:  77.0 , tau:  0.2 , score:  115006.13055760202
bw:  79.0 , tau:  0.2 , score:  114992.05999134232
bw:  81.0 , tau:  0.2 , score:  114970.78871338845
bw:  81.0 , tau:  0.2 , score:  114970.78871338845


In [51]:
X = np.array(data2[["X"]])
Y = np.array(data2[["Y"]])
coords = np.hstack([X,Y])

t = np.array(data2[["time"]])

y = np.array(data2[["Freq"]])

pop = np.array(data2[["pop_density"]])
com = np.array(data2[["commercial_density"]])
roa = np.array(data2[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.model import GTWR

results2 = GTWR(coords, t, y, x, 81.0 , 0.01 , kernel='bisquare').fit()

results2.R2

0.781028247968878

In [36]:
data3 = train2data.sort_values("LocationID")
data3 = data3.reset_index(drop=True)


from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(data3[['pop_density','commercial_density','road_density','X','Y']])
data3[['pop_density','commercial_density','road_density','X','Y']] = scaler.transform(data3[['pop_density','commercial_density','road_density','X','Y']])

In [37]:
X = np.array(data3[["X"]])
Y = np.array(data3[["Y"]])
coords = np.hstack([X,Y])

t = np.array(data3[["time"]])

y = np.array(data3[["Freq"]])

pop = np.array(data3[["pop_density"]])
com = np.array(data3[["commercial_density"]])
roa = np.array(data3[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.sel import search_GTWRparameter

sel = search_GTWRparameter(coords, t, y, x, kernel = 'bisquare')

bw, tau = sel.search(verbose = True)

bw:  6635.0 , tau:  0.2 , score:  145402.96128392537
bw:  4119.0 , tau:  0.2 , score:  145200.79632831912
bw:  2564.0 , tau:  0.2 , score:  144693.11008322184
bw:  1603.0 , tau:  0.2 , score:  143665.1314020664
bw:  1009.0 , tau:  0.2 , score:  141939.9611985844
bw:  642.0 , tau:  0.2 , score:  139763.0805339909
bw:  415.0 , tau:  0.2 , score:  137263.51542975055
bw:  275.0 , tau:  0.2 , score:  135368.8931825211
bw:  188.0 , tau:  0.2 , score:  134345.10558213477
bw:  135.0 , tau:  0.2 , score:  134136.19115951384
bw:  101.0 , tau:  0.2 , score:  133498.67462923465
bw:  81.0 , tau:  0.2 , score:  133249.2893882733
bw:  81.0 , tau:  0.2 , score:  133249.2893882733
bw:  81.0 , tau:  3.8 , score:  133249.2893882733
bw:  81.0 , tau:  0.2 , score:  133249.2893882733
bw:  81.0 , tau:  0.2 , score:  133249.2893882733
bw:  81.0 , tau:  0.2 , score:  133249.2893882733
bw:  81.0 , tau:  0.2 , score:  133249.2893882733


In [52]:
X = np.array(data3[["X"]])
Y = np.array(data3[["Y"]])
coords = np.hstack([X,Y])

t = np.array(data3[["time"]])

y = np.array(data3[["Freq"]])

pop = np.array(data3[["pop_density"]])
com = np.array(data3[["commercial_density"]])
roa = np.array(data3[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.model import GTWR

results3 = GTWR(coords, t, y, x,81.0,0.01, kernel='bisquare').fit()

results3.R2

0.777367711547866

In [46]:
train3data = pd.read_excel("E:\\Capstone Project\\Data\\Taxi_area\\pick202004train3final.xlsx")
train3data = train3data.rename(columns={'Unnamed: 0':'time'})

In [47]:
data4 = train3data.sort_values("LocationID")
data4 = data4.reset_index(drop=True)


from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
scaler.fit(data4[['pop_density','commercial_density','road_density','X','Y']])
data4[['pop_density','commercial_density','road_density','X','Y']] = scaler.transform(data4[['pop_density','commercial_density','road_density','X','Y']])

In [48]:
X = np.array(data4[["X"]])
Y = np.array(data4[["Y"]])
coords = np.hstack([X,Y])

t = np.array(data4[["time"]])

y = np.array(data4[["Freq"]])

pop = np.array(data4[["pop_density"]])
com = np.array(data4[["commercial_density"]])
roa = np.array(data4[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.sel import search_GTWRparameter

sel = search_GTWRparameter(coords, t, y, x, kernel = 'bisquare')

bw, tau = sel.search(verbose = True)

bw:  7628.0 , tau:  0.2 , score:  168065.7404294817
bw:  4733.0 , tau:  0.2 , score:  167888.98680405287
bw:  2943.0 , tau:  0.2 , score:  167439.6381275933
bw:  1838.0 , tau:  0.2 , score:  166419.12874583135
bw:  1154.0 , tau:  0.2 , score:  164679.58018913787
bw:  732.0 , tau:  0.2 , score:  162211.0788252475
bw:  470.0 , tau:  0.2 , score:  159381.10555140363
bw:  309.0 , tau:  0.2 , score:  157196.1179123442
bw:  209.0 , tau:  0.2 , score:  155536.98707965307
bw:  148.0 , tau:  0.2 , score:  155240.3735503658
bw:  109.0 , tau:  0.2 , score:  154708.02008551732
bw:  86.0 , tau:  0.2 , score:  154275.99996506804
bw:  86.0 , tau:  0.2 , score:  154275.99996506804
bw:  86.0 , tau:  0.2 , score:  154275.99996506804
bw:  80.0 , tau:  0.2 , score:  154188.08728755746
bw:  80.0 , tau:  0.2 , score:  154188.08728755746
bw:  80.0 , tau:  0.2 , score:  154188.08728755746
bw:  80.0 , tau:  0.2 , score:  154188.08728755746
bw:  81.0 , tau:  0.2 , score:  154177.08896125937
bw:  81.0 , tau:  0.

In [53]:
X = np.array(data4[["X"]])
Y = np.array(data4[["Y"]])
coords = np.hstack([X,Y])

t = np.array(data4[["time"]])

y = np.array(data4[["Freq"]])

pop = np.array(data4[["pop_density"]])
com = np.array(data4[["commercial_density"]])
roa = np.array(data4[["road_density"]])
x = np.hstack([pop,com,roa])

from gtwr.model import GTWR

results4 = GTWR(coords, t, y, x,81.0,0.01, kernel='bisquare').fit()

results4.R2

0.777507220640079